In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

In [2]:
# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      

In [3]:
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

In [4]:
# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

In [5]:
# 위에까지는 다 똑같음

In [6]:
# station1 data = df1
df1 = df[df['station_nbr'] == 1]

In [7]:
# df 모든 'M' and '-' 을 np.nan으로 형 변환 하는 과정
df1 = df1.apply(pd.to_numeric, errors = 'coerce')

In [8]:
# df1 의 각 column 별로 drop 해야 하는 column 찾은 후, 남은 column에, np.nan값에 그 변수의 mean 값을 넣어 주기.
df1.describe()

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,...,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,1.031190e+05,102897.000000,102897.000000,102564.000000,0.0,102675.000000,102342.000000,102564.000000,102564.000000,0.0,...,102786.000000,103119.000000,103119.000000,103119.000000,103119.0,103119.0,103119.000000,103119.000000,103119.00000,103119.000000
mean,1.367872e+18,60.641855,41.666667,51.427489,NaN,37.944865,45.251627,15.636364,2.063853,NaN,...,6.528402,0.493003,0.500538,0.079656,1.0,1.0,56.000000,0.364909,3.00000,0.285253
std,2.591248e+16,18.975950,16.784905,17.591533,NaN,19.044399,16.126532,15.041496,4.321170,NaN,...,3.247471,0.499953,0.500002,0.270761,0.0,0.0,32.041795,3.609916,2.00001,0.451537
min,1.325376e+18,14.000000,-4.000000,8.000000,NaN,-12.000000,4.000000,0.000000,0.000000,NaN,...,0.600000,0.000000,0.000000,0.000000,1.0,1.0,1.000000,0.000000,0.00000,0.000000
25%,1.345421e+18,45.000000,29.000000,38.000000,NaN,23.000000,33.000000,0.000000,0.000000,NaN,...,4.200000,0.000000,0.000000,0.000000,1.0,1.0,28.000000,0.000000,1.00000,0.000000
50%,1.365552e+18,62.000000,42.000000,53.000000,NaN,39.000000,46.000000,12.000000,0.000000,NaN,...,5.900000,0.000000,1.000000,0.000000,1.0,1.0,56.000000,0.000000,3.00000,0.000000
75%,1.391040e+18,77.000000,56.000000,66.000000,NaN,54.000000,59.000000,27.000000,1.000000,NaN,...,8.200000,1.000000,1.000000,0.000000,1.0,1.0,84.000000,0.000000,5.00000,1.000000
max,1.414714e+18,97.000000,77.000000,86.000000,NaN,71.000000,76.000000,57.000000,21.000000,NaN,...,19.800000,1.000000,1.000000,1.000000,1.0,1.0,111.000000,207.000000,6.00000,1.000000


In [9]:
# Drop 해야 할 Column은 = > drop_columns = ['date', 'depart', 'sunrise', 'sunset', 'codesum'(이미 카테고리화 했기에 drop), 
# 'snowfall', 'store_nbr', 'station_nbr' (여기서 store_nbr가 1밖에 없기에)]
df1_drop_columns = ['date', 'depart', 'sunrise', 'sunset', 'codesum', 'snowfall', 'store_nbr', 'station_nbr']
df1 = df1.drop(columns = df1_drop_columns)

In [10]:
# np.nan값 만 표시된 있는 행이 있는지 확인
df1.describe()

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,item_nbr,units,week7,weekend
count,102897.000000,102897.000000,102564.000000,102675.000000,102342.000000,102564.000000,102564.000000,102564.000000,102786.00000,102231.000000,102675.000000,102675.000000,102786.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.00000,103119.000000
mean,60.641855,41.666667,51.427489,37.944865,45.251627,15.636364,2.063853,0.075666,29.85054,30.016037,5.316649,20.257297,6.528402,0.493003,0.500538,0.079656,56.000000,0.364909,3.00000,0.285253
std,18.975950,16.784905,17.591533,19.044399,16.126532,15.041496,4.321170,0.191589,0.22727,0.227675,3.378105,9.823834,3.247471,0.499953,0.500002,0.270761,32.041795,3.609916,2.00001,0.451537
min,14.000000,-4.000000,8.000000,-12.000000,4.000000,0.000000,0.000000,0.000000,28.95000,29.200000,0.200000,1.000000,0.600000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
25%,45.000000,29.000000,38.000000,23.000000,33.000000,0.000000,0.000000,0.000000,29.70000,29.870000,2.700000,11.000000,4.200000,0.000000,0.000000,0.000000,28.000000,0.000000,1.00000,0.000000
50%,62.000000,42.000000,53.000000,39.000000,46.000000,12.000000,0.000000,0.005000,29.85000,30.020000,4.700000,22.000000,5.900000,0.000000,1.000000,0.000000,56.000000,0.000000,3.00000,0.000000
75%,77.000000,56.000000,66.000000,54.000000,59.000000,27.000000,1.000000,0.030000,30.01000,30.170000,7.400000,28.000000,8.200000,1.000000,1.000000,0.000000,84.000000,0.000000,5.00000,1.000000
max,97.000000,77.000000,86.000000,71.000000,76.000000,57.000000,21.000000,1.980000,30.59000,30.770000,17.900000,36.000000,19.800000,1.000000,1.000000,1.000000,111.000000,207.000000,6.00000,1.000000


In [11]:
df1.isnull().any()

tmax            True
tmin            True
tavg            True
dewpoint        True
wetbulb         True
heat            True
cool            True
preciptotal     True
stnpressure     True
sealevel        True
resultspeed     True
resultdir       True
avgspeed        True
nothing        False
rain           False
other          False
item_nbr       False
units          False
week7          False
weekend        False
dtype: bool

In [12]:
df1.isnull().sum()

tmax           222
tmin           222
tavg           555
dewpoint       444
wetbulb        777
heat           555
cool           555
preciptotal    555
stnpressure    333
sealevel       888
resultspeed    444
resultdir      444
avgspeed       333
nothing          0
rain             0
other            0
item_nbr         0
units            0
week7            0
weekend          0
dtype: int64

In [13]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df1_columns = df1.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df1_columns:
    if (i == 'resultdir'):
        df1[i].fillna(df1[i].mode()[0], inplace=True)
        print(df1[i].mode()[0])
    else:
        df1[i].fillna(df1[i].mean(), inplace=True)

21.0


In [14]:
df1.describe()

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,item_nbr,units,week7,weekend
count,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.000000,103119.00000,103119.000000
mean,60.641855,41.666667,51.427489,37.944865,45.251627,15.636364,2.063853,0.075666,29.850540,30.016037,5.316649,20.260495,6.528402,0.493003,0.500538,0.079656,56.000000,0.364909,3.00000,0.285253
std,18.955512,16.766828,17.544129,19.003355,16.065660,15.000963,4.309526,0.191073,0.226903,0.226693,3.370825,9.802782,3.242223,0.499953,0.500002,0.270761,32.041795,3.609916,2.00001,0.451537
min,14.000000,-4.000000,8.000000,-12.000000,4.000000,0.000000,0.000000,0.000000,28.950000,29.200000,0.200000,1.000000,0.600000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
25%,45.000000,29.000000,38.000000,24.000000,33.000000,0.000000,0.000000,0.000000,29.700000,29.870000,2.700000,11.000000,4.200000,0.000000,0.000000,0.000000,28.000000,0.000000,1.00000,0.000000
50%,62.000000,42.000000,53.000000,39.000000,46.000000,12.000000,0.000000,0.005000,29.850000,30.020000,4.700000,22.000000,6.000000,0.000000,1.000000,0.000000,56.000000,0.000000,3.00000,0.000000
75%,77.000000,56.000000,66.000000,54.000000,59.000000,27.000000,1.000000,0.030000,30.010000,30.170000,7.300000,28.000000,8.200000,1.000000,1.000000,0.000000,84.000000,0.000000,5.00000,1.000000
max,97.000000,77.000000,86.000000,71.000000,76.000000,57.000000,21.000000,1.980000,30.590000,30.770000,17.900000,36.000000,19.800000,1.000000,1.000000,1.000000,111.000000,207.000000,6.00000,1.000000


In [23]:
# 모든 형변환 완료되었는지 확인하기.
df1.dtypes

tmax           float64
tmin           float64
tavg           float64
dewpoint       float64
wetbulb        float64
heat           float64
cool           float64
preciptotal    float64
stnpressure    float64
sealevel       float64
resultspeed    float64
resultdir      float64
avgspeed       float64
nothing          int64
rain             int64
other            int64
item_nbr         int64
units            int64
week7            int64
weekend          int64
dtype: object

In [15]:
df1.isnull().any()

tmax           False
tmin           False
tavg           False
dewpoint       False
wetbulb        False
heat           False
cool           False
preciptotal    False
stnpressure    False
sealevel       False
resultspeed    False
resultdir      False
avgspeed       False
nothing        False
rain           False
other          False
item_nbr       False
units          False
week7          False
weekend        False
dtype: bool

In [17]:
# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df1['relative_humility'] = 100*(np.exp((17.625*((df1['dewpoint']-32)/1.8))/(243.04+((df1['dewpoint']-32)/1.8)))/np.exp((17.625*((df1['tavg']-32)/1.8))/(243.04+((df1['tavg']-32)/1.8))))

# 체감온도 계산
df1["windchill"] = 35.74 + 0.6215*df1["tavg"] - 35.75*(df1["avgspeed"]**0.16) + 0.4275*df1["tavg"]*(df1["avgspeed"]**0.16)

In [18]:
# Station1에 대한 전처리 완료 
df1.columns

Index(['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'nothing', 'rain', 'other', 'item_nbr', 'units', 'week7',
       'weekend', 'relative_humility', 'windchill'],
      dtype='object')

In [ ]:
model1 = sm.OLS.from_formula('np.log(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + C(resultdir) + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data=df1)
result1 = model1.fit()
print(result1.summary())

In [61]:
df1['item_nbr'][df1['units'] != 0].unique()

array([ 9, 28, 51, 40, 93, 89, 99, 47], dtype=int64)

In [62]:
# resultdir이 Category가 아닐 때
model1_2 = sm.OLS.from_formula('units ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data=df1)

result1_2 = model1_2.fit()

print(result1_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     1250.
Date:                Tue, 03 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:17:28   Log-Likelihood:            -2.2914e+05
No. Observations:              103119   AIC:                         4.586e+05
Df Residuals:                  102985   BIC:                         4.598e+05
Df Model:                         133                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]          0.0069      0

In [83]:
sm.stats.anova_lm(result1_2, typ=2)

,sum_sq,df,F,PR(>F)
C(nothing),2.298794,2.0,0.230295,7.942994e-01
C(rain),0.478330,1.0,0.095839,7.568819e-01
C(other),0.002502,1.0,0.000501,9.821363e-01
C(item_nbr),828775.864370,110.0,1509.591500,0.000000e+00
C(week7),735.808075,6.0,24.571311,2.844275e-29
C(weekend),281.838260,1.0,56.469635,5.753195e-14
tmax,16.723764,1.0,3.350804,6.717507e-02
tmin,2.885636,1.0,0.578171,4.470317e-01
tavg,0.068091,1.0,0.013643,9.070170e-01
dewpoint,13.460708,1.0,2.697012,1.005395e-01


In [86]:
anova_result1_2 = sm.stats.anova_lm(result1_2, typ=2).sort_values(by=['PR(>F)'], ascending = False)

In [87]:
anova_result1_2.columns

Index(['sum_sq', 'df', 'F', 'PR(>F)'], dtype='object')

In [89]:
anova_result1_2[anova_result1_2['PR(>F)'] <= 0.05]

,sum_sq,df,F,PR(>F)
windchill,22.649665,1.0,4.538129,3.315017e-02
C(weekend),281.838260,1.0,56.469635,5.753195e-14
C(week7),735.808075,6.0,24.571311,2.844275e-29
C(item_nbr),828775.864370,110.0,1509.591500,0.000000e+00


In [90]:
final_model1_2 = sm.OLS.from_formula('units ~ windchill + C(weekend) + C(week7) + C(item_nbr) + 0', data=df1)

final_result1_2 = final_model1_2.fit()

print(final_result1_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     1420.
Date:                Tue, 03 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:58:03   Log-Likelihood:            -2.2916e+05
No. Observations:              103119   AIC:                         4.586e+05
Df Residuals:                  103001   BIC:                         4.597e+05
Df Model:                         117                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(weekend)[0]          0.0115      0

## Item_nbr 없는 것은 삭제 후, 다시 돌리기 (카테고리 재정립)

In [42]:
## 여기서, 안팔린 item을 제외해고 다시 돌려보자.
df1_sorted = df1[df1['units'] != 0]
# Data의 갯수가 너무 많이 줄어 들기는 함.

In [43]:
df1_sorted.describe()

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,...,avgspeed,nothing,rain,other,item_nbr,units,week7,weekend,relative_humility,windchill
count,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,...,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000
mean,59.041757,40.364037,49.967619,36.809256,44.070053,16.898569,1.866188,0.074805,29.855188,30.021091,...,6.481086,0.476886,0.515468,0.081682,36.500174,13.079249,3.061522,0.296489,61.845630,47.553263
std,19.198233,16.955794,17.775544,19.142099,16.237926,15.377767,4.093847,0.188912,0.230669,0.230732,...,3.277740,0.499552,0.499848,0.273928,28.879812,17.346124,2.000010,0.456789,14.650134,21.651034
min,14.000000,-4.000000,8.000000,-12.000000,4.000000,0.000000,0.000000,0.000000,28.950000,29.200000,...,0.600000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,20.389775,-8.141708
25%,43.000000,28.000000,36.000000,23.000000,32.000000,0.000000,0.000000,0.000000,29.700000,29.880000,...,4.100000,0.000000,0.000000,0.000000,9.000000,3.000000,1.000000,0.000000,51.851412,31.440103
50%,60.000000,40.000000,51.000000,38.000000,45.000000,14.000000,0.000000,0.005000,29.860000,30.030000,...,5.900000,0.000000,1.000000,0.000000,28.000000,6.000000,3.000000,0.000000,61.659031,49.118262
75%,75.000000,54.000000,65.000000,53.000000,58.000000,29.000000,1.000000,0.030000,30.010000,30.180000,...,8.200000,1.000000,1.000000,0.000000,47.000000,15.000000,5.000000,1.000000,71.931829,66.008573
max,97.000000,77.000000,86.000000,71.000000,76.000000,57.000000,21.000000,1.980000,30.590000,30.770000,...,19.800000,1.000000,1.000000,1.000000,99.000000,207.000000,6.000000,1.000000,100.000000,90.656121


In [44]:
# Sort하고, resultdir이 Category일 때
model1_sorted = sm.OLS.from_formula('units ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + C(resultdir) + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data=df1_sorted)

result1_sorted = model1_sorted.fit()

print(result1_sorted.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     42.00
Date:                Tue, 03 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:36:53   Log-Likelihood:                -11326.
No. Observations:                2877   AIC:                         2.278e+04
Df Residuals:                    2812   BIC:                         2.317e+04
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]     -0.9413      1

In [79]:
# Sort하고, resultdir이 Category가 아닐 때
model1_sorted_2 = sm.OLS.from_formula('units ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data=df1_sorted)

result1_sorted_2 = model1_sorted_2.fit()

print(result1_sorted_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     87.96
Date:                Tue, 03 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:36:26   Log-Likelihood:                -11347.
No. Observations:                2877   AIC:                         2.276e+04
Df Residuals:                    2846   BIC:                         2.294e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]        -0.3334      1.44

In [82]:
(sm.stats.anova_lm(result1_sorted_2, typ=2))

,sum_sq,df,F,PR(>F)
C(nothing),18.517325,2.0,0.058683,9.430072e-01
C(rain),0.424757,1.0,0.002692,9.586231e-01
C(other),15.120111,1.0,0.095833,7.569111e-01
C(item_nbr),388046.770630,7.0,351.355622,0.000000e+00
C(week7),24232.939200,6.0,25.598571,8.976137e-30
C(weekend),9023.154532,1.0,57.189891,5.306169e-14
tmax,872.194373,1.0,5.528078,1.878150e-02
tmin,251.898101,1.0,1.596562,2.064946e-01
tavg,100.104750,1.0,0.634477,4.257845e-01
dewpoint,485.689346,1.0,3.078360,7.944808e-02


In [ ]:
# 버려야 할 목록:
# C(nothing), 

In [ ]:
C(nothing)[0]        -0.3334      1.444     -0.231      0.817      -3.166       2.499
C(nothing)[1]         0.3255      1.443      0.226      0.822      -2.503       3.154
C(rain)[T.1]         -0.1494      2.880     -0.052      0.959      -5.796       5.497
C(other)[T.1]         0.2985      0.964      0.310      0.757      -1.592       2.189
C(item_nbr)[T.28]   -24.3897      0.589    -41.423      0.000     -25.544     -23.235
C(item_nbr)[T.40]   -28.8742      1.273    -22.678      0.000     -31.371     -26.378
C(item_nbr)[T.47]   -21.5807      0.839    -25.720      0.000     -23.226     -19.935
C(item_nbr)[T.51]   -25.6846      1.045    -24.589      0.000     -27.733     -23.636
C(item_nbr)[T.89]   -28.1174      1.508    -18.643      0.000     -31.075     -25.160
C(item_nbr)[T.93]   -23.9907      1.008    -23.790      0.000     -25.968     -22.013
C(item_nbr)[T.99]   -27.8399      1.003    -27.755      0.000     -29.807     -25.873
C(week7)[T.1]        -0.3812      0.902     -0.423      0.672      -2.149       1.387
C(week7)[T.2]        -1.5010      0.904     -1.660      0.097      -3.274       0.272
C(week7)[T.3]        -1.3501      0.892     -1.513      0.130      -3.100       0.399
C(week7)[T.4]         1.3383      0.882      1.518      0.129      -0.390       3.067
C(week7)[T.5]         1.6348      0.502      3.257      0.001       0.650       2.619
C(week7)[T.6]         2.2351      0.504      4.436      0.000       1.247       3.223
C(weekend)[T.1]       3.8700      0.512      7.562      0.000       2.867       4.873